# Import

In [2]:
from tqdm import tqdm_notebook
import deepdish as dd
import numpy as np
import os
import h5py
import pandas as pd
from scipy import stats
from _searchlight_util import *
import matplotlib.pyplot as plt
from _classification_util import Convert_SourceItem_to_PairedItem_Order
from _mempal_util import room_obj_assignments, save_obj, load_obj, create_dirs, subject_ids

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']


In [3]:
### CHECK IF SLs were generated
################################


# roi = 'SL'
# nSubj =25


# # date = 20230802; shift = 4; win_size=9 ;
# # date = 20240108; shift = 4; win_size=9 ;
# date = 20240401; shift = 4; win_size=9 ;


# error_rois = {}
# full_list = []
# for betatype in ['rooms', 'objects']:#'objects']: #['rooms','objects']:
#     error_rois[betatype] = {}
#     for hem in ['L','R']:
#         hem_label = hem
#         betatype_label = betatype
#         error_rois[betatype][hem] = []
#         for roi_id in tqdm_notebook(range(len(SLlist[hem]))):

#             dirpath = '../PythonData2024/Output/ClassifyRecalls'
#             fname = '{}_{}{:03d}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls'.format(
#                 date,
#                 roi,
#                 roi_id,
#                 hem_label,
#                 betatype_label,
#                 win_size,
#                 shift) + '.h5'
            
#             fullpath = os.path.join(dirpath,fname)

#             if os.path.exists(fullpath):
#                 continue
#             else:
#                 error_rois[betatype][hem].append(roi_id)
#                 full_list.append(roi_id)



/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1483 [00:00<?, ?it/s]

  0%|          | 0/1483 [00:00<?, ?it/s]

  0%|          | 0/1483 [00:00<?, ?it/s]

  0%|          | 0/1483 [00:00<?, ?it/s]

In [3]:
def GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath = '../PythonData2024/Output/ClassifyRecalls'):
    '''
    - grabs the whole-brain results of classifying recalls (GRs+FRs)
    - returns 'scores' which is a dict with the data in measure_list
    
    '''

    trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']

    nPerm = 1000
    nTrials = len(trials)
    nItems   = 23

    
    scores = {}

    for betatype in tqdm_notebook(betatypes):
        scores[betatype] = {}

        print("...BETATYPE: ", betatype)

        for measuretype in tqdm_notebook(measure_list):
            scores[betatype][measuretype] = {}

            print("............MEASURE: ", measuretype)

            for hem in ['R','L']:
                print("...............HEM: ",hem)

                hem_label = hem
                betatype_label=betatype

                ###
                ### Specific initiation for different array shapes
                ###

                if 'RIP_window' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),win_size,nTrials,nSubj))) 

                if 'evidence_window' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),win_size,nItems, nTrials,nSubj))) 

                if 'accuracy' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),nTrials,nSubj,nPerm+1)))

    #             if 'real_minus_prior' == measuretype:
    #                 scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),nTrials,nSubj)))   

                if 'evidence' == measuretype:
                    scores[betatype][measuretype][hem] = {}

                if 'conf_mat' == measuretype: # comes in as (nItems,nItems,nTrials,nSubj,nPerm+1)
                    scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems,nTrials,nSubj), fill_value=np.nan)
#                     scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems), fill_value=np.nan)


                if 'RIPs' == measuretype:
    #                 scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nItems,nTrials,nSubj), fill_value=np.nan)
                    scores[betatype][measuretype][hem] = np.full((len(SLlist[hem]),nItems,nTrials,nSubj), fill_value=np.nan)
                    


                if 'avg_RIPs' == measuretype:
                    scores[betatype][measuretype][hem] = np.zeros(((len(SLlist[hem]),2, nItems,nSubj))) # for GR and FR
                ###
                ### Load the saved .h5 files
                ###

                for roi_id in tqdm_notebook(range(len(SLlist[hem]))[:]):

#                     dirpath = '../PythonData2024/Output/ClassifyRecalls'

                    fname = '{}_{}{:03d}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls'.format(
                    date,
                    roi,
                    roi_id,
                    hem_label,
                    betatype_label,
                    win_size,
                    shift) + '.h5'

                    fullpath = os.path.join(dirpath,fname)

                    temp = dd.io.load(fullpath,'/{}/{}/{}'.format(betatype,hem,measuretype))

                    if measuretype=='conf_mat':
#                         print('.......conf_mat shape: ', temp.shape)
#                         scores[betatype][measuretype][hem][roi_id] = np.nanmean(np.nanmean(temp[:,:,:,:,0],2),2)
                        scores[betatype][measuretype][hem][roi_id,:,:,:,:] = temp[:,:,:,:,0] # permutations are empty, so no need to collect them
#                          scores[betatype][measuretype][hem][roi_id,:,:] = temp[:,:,:,:,0].sum(-1).sum(-1) # permutations are empty, so no need to collect them

                        
                    elif measuretype == "avg_RIPs":
                        scores[betatype][measuretype][hem][roi_id,0,:,:] = temp['GR']
                        scores[betatype][measuretype][hem][roi_id,1,:,:] = temp['FR']
                    else:
                        scores[betatype][measuretype][hem][roi_id] = temp
                        
                        
    return scores

                


In [4]:
## 
## IMPORT
##

#accuracy[betatype][hem]= np.full((nTrials,nSubj,nPerm+1),fill_value=np.nan)
# conf_mat[betatype][hem]= np.full((nItems,nItems,nTrials,nSubj,nPerm+1), fill_value=np.nan)
# evidence[betatype][hem]= {} #dict of trials and subjects
# cp_window[betatype][hem]= np.full((win_size,nTrials,nSubj), fill_value=np.nan) 
# trial_length[betatype][hem] = np.full((nTrials,nSubj),fill_value=np.nan)

dirpath = '../PythonData2024/Output/ClassifyRecalls'


date = 20240401; shift = 4; win_size=9 ;


# betatypes = ['rooms','objects']
betatypes = ['objects', 'rooms'] #['objects']
roi = 'SL'
measure_list = [
#     'conf_mat', #  # only acquire if doing the accurazy brainmap not necessary for the ROCN evidence
    "avg_RIPs", 
#     'evidence',
    'accuracy', #
#     'RIP_window', # 
    'RIPs',
#     'conf_mat', # 
  ]

scores = GetRecallEvidence(date, roi, shift, win_size, measure_list, betatypes, dirpath)

     

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/2 [00:00<?, ?it/s]

...BETATYPE:  objects


/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/1483 [00:00<?, ?it/s]

............MEASURE:  RIPs
...............HEM:  R


  0%|          | 0/1483 [00:00<?, ?it/s]

...............HEM:  L


  0%|          | 0/1483 [00:00<?, ?it/s]

In [29]:
# ## SAVE CONFMAT since it takes a long time to load up

# date = 20240401; shift = 4; win_size=9 ;

# fname = '{}_{}_hems{}_betas{}_winsize{}_shift{}_ClassifyRecalls_CONFMAT'.format(
# date,
# roi,
# 'both',
# 'both',
# win_size,
# shift) + '.pkl'

# conf_mat_scores = {}
# for betatype in tqdm_notebook(betatypes):
#     conf_mat_scores[betatype] = {}
#     for hem in ['R','L']:
#         conf_mat_scores[betatype][hem] = scores[betatype]['conf_mat'][hem].copy()
        
# save_obj("../PythonData2024/Output/confmats/{}".format(fname),conf_mat_scores)  

# print("...confmats saved.")
    


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2 [00:00<?, ?it/s]

...confmats saved.


# 1. Create Recall Classifier Accuracy Brainmap

In [76]:

#scores --> nPerm+1
#ev_prank --> nPerm+1

nv = 40962 #verts in fsaverage6 brain
# raw_vox = {} # raw values
# p_vox = {} # non-parametric p-values
# q_vox = {} # q-values from FDR-correction
# z_vox = {} # z-values

#     'cp_window', # (nSLs, 9,11,25)
#     'evidence_window', # (nSLs, 9,23,11,25)
#     'accuracy', #(nSLs,11,25,1001)
#     'real_minus_prior', #(nSLs,11,25)

chance = 1/23

trials = ['GR0', 'GR1', 'GR2', 'GR3','GR4', 'GR5', 'GR6', 'GR7','GR8', 'GR9', 'GR10','FR']
nPerm = 1000

# trial_to_use = 'GR'
# trials_to_use
measures = {}
for trial_to_use in ['GR','FR']:
    
    ## which type of recall are we going to get accuracy from?
    ## choose between guided recalls (GR) or free recalls (FR)
    trials_to_use = np.arange(len(trials))[:-1] if trial_to_use == 'GR' else [np.arange(len(trials))[-1]]

    measures[trial_to_use] = {}
    for betatype in ['objects','rooms']:# ['rooms','objects']:
        measures[trial_to_use][betatype] = {}
        
        for measuretype in ['accuracy','RIPs','avg_RIPs','conf_mat']: #measure_list:
            measures[trial_to_use][betatype][measuretype] = {}

            if 'accuracy' == measuretype:

                print("....first if: {} {}".format(betatype, measuretype))

                dd_hem = {"L":np.nanmean(np.nanmean(scores[betatype][measuretype]['L'][:,trials_to_use,:,:],1),1),
                          "R": np.nanmean(np.nanmean(scores[betatype][measuretype]['R'][:,trials_to_use,:,:],1),1)} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}

                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)
                

            ### SHOULD REMOVE THIS, 20230812 DONT NEED IT
            if 'RIPs' == measuretype or "avg_RIPs" == measuretype:
                t_vox = {}
                t_p_vox = {}
                raw_vox = {}

                if 'RIPs'==measuretype:
                    dd_hem = {"L":np.nanmean(np.nanmean(scores[betatype][measuretype]['L'][:,:,trials_to_use,:],1),1),
                          "R": np.nanmean(np.nanmean(scores[betatype][measuretype]['R'][:,:,trials_to_use,:],1),1)} #shape (nSLs,nSubj)

                elif "avg_RIPs" == measuretype:
                    idx2use = 0 if trial_to_use=='GR' else 1
                    #scores[betatype]['avg_RIPs']['R'].shape #(1483, 2, 23, 25)
                    dd_hem = {"L": np.nanmean(scores[betatype][measuretype]['L'][:,idx2use,:,:],1),
                             "R": np.nanmean(scores[betatype][measuretype]['R'][:,idx2use,:,:],1)} #shape (nSLs,nSubj)
                
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                for hem in ['L','R']:
                    t_vox[hem] = np.full((nv),fill_value=np.nan)
                    t_p_vox[hem] = np.full((nv),fill_value=np.nan)
                    raw_vox[hem] = np.nanmean(dd_vox[hem],1) #avg out subj
                    for vi in tqdm_notebook(range(nv)):

                        # get t value from 25 subj numbers for every vertex
    #                     t, p = stats.ttest_1samp(dd_vox[hem][vi,:],chance,alternative='greater') #one sided t-test 
                        t, p = stats.ttest_1samp(dd_vox[hem][vi],chance) #only two-sided available
                        p = p / 2 # make it 1 sided
                        t_vox[hem][vi] = t
                        t_p_vox[hem][vi] = p

                measures[trial_to_use][betatype][measuretype]['t_q'] = FDR_p_hem(t_p_vox)
                measures[trial_to_use][betatype][measuretype]['t'] = t_vox
                measures[trial_to_use][betatype][measuretype]['raw'] = raw_vox
                
            if 'conf_mat' == measuretype: #accuracy based on confmat
                
                confmat_acc_perm = {}
                for hem in ['L','R']:
                    confmat_acc_perm[hem] = np.full((len(SLlist[hem]), nPerm+1), fill_value=np.nan)
                    confmat = scores[betatype]['conf_mat'][hem][:,:,:,trials_to_use,:].sum(-1).sum(-1) # shape (nSLs, 23,23)

                    ## PERMUTATION:
                    print('...runing confmat permutation.')
                    for sl in tqdm_notebook(range(len(SLlist[hem]))):
                        for p in range(nPerm+1):
                            if p > 0:
                                permconf = confmat[sl,np.random.permutation(np.arange(23)),:].copy()
                            else:
                                permconf = confmat[sl].copy()

                            confmat_acc_perm[hem][sl,p] = np.diag(permconf).sum() / permconf.sum()
                            
                    
                dd_hem = {"L":confmat_acc_perm['L'],"R": confmat_acc_perm['R']} #shape (nSLs,nPerm+1)
                dd_vox = SLtoVox(dd_hem, ROIlist,nv,zeronan=False) #

                ## z-score
                measures[trial_to_use][betatype][measuretype]['raw'] = {}
                measures[trial_to_use][betatype][measuretype]['z'] = {}
                
                dd_vox_p = {}
                for hem in ['L','R']:
                    dd_vox_p[hem] = NonparametricP(dd_vox[hem],sided=1) #accuracy can only be positive so 1-sided
                    measures[trial_to_use][betatype][measuretype]['raw'][hem] = dd_vox[hem][:,0]
                    measures[trial_to_use][betatype][measuretype]['z'][hem] = nullZ(dd_vox[hem])

                measures[trial_to_use][betatype][measuretype]['p'] = dd_vox_p
                measures[trial_to_use][betatype][measuretype]['q'] = FDR_p_hem(dd_vox_p)

                


###
### SAVE SAVE SAVE
###
# date = 20230814 # after re-running classifyrecalls on objects
# date = 20240108 # 
date = 20240401 # second permutation fix, keep structure intact

dirpath = '../PythonData2024/Output/brainmaps/'
fname = '{}_ClassifyRecalls_Accuracy_shift{}'.format(date,shift) + '.pkl'
fullpath = os.path.join(dirpath,fname)

#     data = measures
save_obj(fullpath,measures)
print('...saving in: ',fullpath)


print('complete.')

....first if: objects accuracy


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice
/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/ipykernel_launcher.py:80: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/40962 [00:00<?, ?it/s]

  0%|          | 0/40962 [00:00<?, ?it/s]

  0%|          | 0/40962 [00:00<?, ?it/s]

...

...runing confmat permutation.


  0%|          | 0/1483 [00:00<?, ?it/s]

...runing confmat permutation.


  0%|          | 0/1483 [00:00<?, ?it/s]

...saving in:  ../PythonData2024/Output/brainmaps/20240401_ClassifyRecalls_Accuracy_shift4.pkl
complete.


# 2. Get ROCN and POCN Item Evidence for Recalls

use the ROCN and POCN's as ROI masks to extract item evidence from recalls

In [8]:


##########
########## Grab the ROCN and POCN networks (made in 03b notebook)
##########

# date = 20240108
date = 20240401 # second permutation fix


besttype = 'Z'
shift = 4
top_thresh = 50

superteam_path = '../PythonData2024/Output/Networks'


superteam_SL_savename = '{}_Networks_Best{}SLs_shift{}_top{}.pkl'.format(date,besttype,shift,top_thresh)
path = os.path.join(superteam_path,superteam_SL_savename)
best_Z_SLs = load_obj(path)



In [9]:
# Retrieved Object Classifier Network (ROCN)
ROCN_SLs = best_Z_SLs['objects']['rooms']

# Perceived Object Classifier Network (POCN)
POCN_SLs = best_Z_SLs['objects']['objects']

# RRCN_SLs = best_Z_SLs['rooms']['objects']
# PRCN_SLs = best_Z_SLs['rooms']['rooms']

In [89]:
nItems = 23

useAvgRIPs = True # use norm across all trials and subjects instead of norm across trials (normal RIPs)

superteam_SLs = best_Z_SLs

betatype = 'objects'

modified_rip = {} # 

item_order = np.arange(nItems)

for network, SLs2Use in zip(("ROCN","POCN"),(ROCN_SLs, POCN_SLs)):
    
    modified_rip[network] = {}
    
    ## ROCN is when we classify an item that is not visible
    ## POCN is when we classify an item that is visible (perceived)
    
    videotype = 'rooms' if network=='ROCN' else 'objects'
    
    print('betatype: {} | videotype: {}'.format(betatype,videotype))
        
    bestL = superteam_SLs[betatype][videotype]['L'].astype(int)
    bestR = superteam_SLs[betatype][videotype]['R'].astype(int)
    
    print(network)
    
    print('separate', ROCN_SLs['L'])
    print('embedded', bestL )
    
    if useAvgRIPs==True:
        concat_rip = np.concatenate((scores[betatype]['avg_RIPs']['L'][bestL],
                                scores[betatype]['avg_RIPs']['R'][bestR]),axis=0)
    else:
        concat_rip = np.concatenate((scores[betatype]['RIPs']['L'][bestL],
                                scores[betatype]['RIPs']['R'][bestR]),axis=0)
        
    print(concat_rip.shape)
    
    for trial_to_use in ['GR','FR']:

        modified_rip[network][trial_to_use] = np.full((nItems,nSubj),fill_value=np.nan)

        ## which type of recall are we going to get accuracy from?
        ## choose between guided recalls (GR) or free recalls (FR)
        
        if useAvgRIPs==False:
            # shape of concat_rip = (50, 12, 23, 25)
            trials_to_use = np.arange(len(trials))[:-1] if trial_to_use == 'GR' else [np.arange(len(trials))[-1]]
            current_rip = concat_rip[:,:,trials_to_use,:]
            current_rip = np.nanmean(current_rip,2) #average out trials
            current_rip = np.nanmean(current_rip,0) # average out SLs, now it's (23,25)
        else:
            # shape of concat_rip = (50, 2, 23, 25)
            trials_to_use = 0 if trial_to_use == 'GR' else 1
            current_rip = concat_rip[:,trials_to_use,:,:]
            current_rip = np.nanmean(current_rip,0) # average out SLs, now it's (23,25)
            print(current_rip.shape)
        
        print(trials_to_use)

        for si, subj in enumerate(subject_ids):

            if betatype == 'rooms':
                #no need to re-order
                print("no need to re-order ")
                new_item_order = item_order

            elif betatype == 'objects':

                # need to re-order so that object index is equal to room index per subject
                new_item_order = Convert_SourceItem_to_PairedItem_Order(subj,item_order,room_obj_assignments,fromItem='objidx',toItem='roomidx')
                
#                 print(new_item_order)
                      
            modified_rip[network][trial_to_use][:,si] = current_rip[new_item_order,si]



betatype: objects | videotype: rooms
ROCN
separate [ 252 1410  154  469 1408 1187  985  240 1312  618  825  810  590  949
 1429 1441  956  714  372 1277   57 1397  527  745  461  172]
embedded [ 252 1410  154  469 1408 1187  985  240 1312  618  825  810  590  949
 1429 1441  956  714  372 1277   57 1397  527  745  461  172]
(50, 2, 23, 25)
(23, 25)
0


/jukebox/pkgs/PYGER/0.9.1/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Mean of empty slice


(23, 25)
1
betatype: objects | videotype: objects
POCN
separate [ 252 1410  154  469 1408 1187  985  240 1312  618  825  810  590  949
 1429 1441  956  714  372 1277   57 1397  527  745  461  172]
embedded [ 802 1419  360  775  564  818  206 1004 1119  359  264 1141  290  485
  777]
(50, 2, 23, 25)
(23, 25)
0
(23, 25)
1


In [107]:
useAvgRIPs

True

In [11]:

###
### SAVE SAVE SAVE SAVE SAVE
###

# date = 20240108 # 
date = 20240401 # second permutation fix
savedir = '../PythonData2024/Output/RecallEvidence' ; create_dirs(savedir)

fname = '{}_NetworksRecallEvidence_shift{}_top{}'.format(date,shift,top_thresh) + '.pkl'
                                                                                                
save_obj(os.path.join(savedir,fname),modified_rip)
print('saved... ')
print('...', os.path.join(savedir,fname))

Folder already exists: ../PythonData2024/Output/RecallEvidence
Folder already exists: ../PythonData2024/Output/20240916_RecallEvidence
Folder and its parents created at: ../PythonData2024/Output/20240917_RecallEvidence
saved... 
... ../PythonData2024/Output/20240917_RecallEvidence/20240917_NetworksRecallEvidence_shift4_top50.pkl
